In [20]:
import numpy as np
import tensorflow as tf

In [24]:
ct = np.arange(7*5*2).reshape(7,5,2)
np.transpose(ct).shape
# m*n -> transpose -> n*m
# i*j*k -> transpose -> ikj,jik,jki,kij,kji
np.transpose(ct, [1,2,0]).shape

(5, 2, 7)

In [26]:
sess = tf.Session()

In [28]:
t = [i for i in range(1,25)]
t = tf.reshape(t,[2,3,4])
t = t[-1]
print(sess.run(t))

[[13 14 15 16]
 [17 18 19 20]
 [21 22 23 24]]


In [67]:
#case study 1 : 4글자로 구성된 단어 -> 학습
#단어의 앞 3글자 입력 -> 마지막 글자 예측
# wood -> 학습 -> 모델
# woo 입력 ------>모델-> d 예측
# wop 입력 ------>모델-> d 예측
char_arr=[]   #'a':97~'z':122, chr(97)->'a'
for i in range(97,123):
    char_arr.append(chr(i))
    
num_dic = {n:i for i, n in enumerate(char_arr) }    
#{'a':0,'b':1,...'z':25}
dic_len = len(num_dic)

In [68]:
seq_data = ['word','wood','deep','dive',
            'cold','cool','load','love',
            'kiss','kind']
#x : wor, y : d
#x : woo, y : d

In [69]:
def make_batch(seq_data):
    input_batch=[]
    target_batch=[]
    for seq in seq_data: #seq = 'word'
        myInput = [num_dic[n] for n in seq[:-1]] # wor => 22,14,17
        #print(myInput)
        target = num_dic[seq[-1]]
        #원핫 인코딩
        input_batch.append(np.eye(dic_len)[myInput])
        target_batch.append(target)
    
    return input_batch, target_batch

In [70]:
input_batch, target_batch = make_batch(seq_data)
# woo, d

In [71]:
np.shape(input_batch) #(10, 3, 26)
#10:단어개수, 3:3글자, 26:26가지 종류

(10, 3, 26)

In [72]:
#softmax_cross_entropy_with_logits함수는
#label값을 one-hot 인코딩으로 넘겨줌
#sparse_softmax_cross_entropy_with_logits함수는
#label값이 one-hot 인코딩이 안된 경우에 사용

In [73]:
lr=0.01
n_hidden=128   #셀 출력 개수
total_epoch=30 #에폭
n_step=3  #입력 3글자
n_input = n_class = dic_len 
#26=입력 크기(n_input)=클래스 종류 개수(n_class)

In [74]:
tf.reset_default_graph()

In [75]:
#신경망 모델 구성
x = tf.placeholder(tf.float32,[None,n_step,n_input])
#None:전체 단어수, n_step:3글자 입력, n_input:26개 종류
y = tf.placeholder(tf.int32,[None]) # => [3], [5], ...
#y가 원핫 인코딩 되어 있다면-> [None,n_class]
w = tf.Variable(tf.random_normal([n_hidden,n_class]))
b = tf.Variable(tf.random_normal([n_class]))
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1,cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell,x, dtype=tf.float32)

In [76]:
#outputs shape : (10, 3, 128)
outputs = tf.transpose(outputs, [1,0,2])
#(10, 3, 128) => (3, 10, 128)
outputs = outputs[-1] # (10, 128)
model = tf.matmul(outputs,w)+b
#tf.matmul([10, 128],[128, 26]) => [10,26]

In [77]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=y))
opt = tf.train.AdamOptimizer(lr).minimize(cost)

In [78]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [79]:
#outputs을 실행하여 outputs의 shape 출력?
print(sess.run(outputs, feed_dict={x:input_batch}).shape)

(10, 128)


In [80]:
for epoch in range(total_epoch):
    _,cv = sess.run([opt,cost], feed_dict={x:input_batch, y:target_batch})
    print("에폭:", "%04d" %(epoch+1),"비용:","{:.5f}".format(cv))
print("모델 작성 완료")

에폭: 0001 비용: 4.68996
에폭: 0002 비용: 3.62536
에폭: 0003 비용: 2.04646
에폭: 0004 비용: 1.80792
에폭: 0005 비용: 0.93985
에폭: 0006 비용: 0.69468
에폭: 0007 비용: 0.68095
에폭: 0008 비용: 0.73747
에폭: 0009 비용: 0.53959
에폭: 0010 비용: 0.25656
에폭: 0011 비용: 0.13384
에폭: 0012 비용: 0.39243
에폭: 0013 비용: 0.24359
에폭: 0014 비용: 0.12168
에폭: 0015 비용: 0.12432
에폭: 0016 비용: 0.06525
에폭: 0017 비용: 0.48709
에폭: 0018 비용: 0.19150
에폭: 0019 비용: 0.08338
에폭: 0020 비용: 0.13685
에폭: 0021 비용: 0.03724
에폭: 0022 비용: 0.08870
에폭: 0023 비용: 0.41516
에폭: 0024 비용: 0.21761
에폭: 0025 비용: 0.23562
에폭: 0026 비용: 0.03382
에폭: 0027 비용: 0.05721
에폭: 0028 비용: 0.03020
에폭: 0029 비용: 0.02197
에폭: 0030 비용: 0.15487
모델 작성 완료


In [81]:
pred = tf.cast(tf.argmax(model,1),tf.int32)
predCheck = tf.equal(pred,y)
accuracy = tf.reduce_mean(tf.cast(predCheck,tf.float32))

input_batch, target_batch = make_batch(seq_data)
pv, av = sess.run([pred, accuracy], feed_dict={x:input_batch, y:target_batch})

In [84]:
predict_words=[]
for i, v in enumerate(seq_data):
    last_char = char_arr[pv[i]]
    predict_words.append(v[:3]+last_char)
print("예측결과\n")
print("입력값:",[w[:3] for w in seq_data])
print("예측값:", predict_words)
print("정확도:", av)

예측결과

입력값: ['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis', 'kin']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0
